Imports

In [ ]:
#imports
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, set_seed
from datasets import Dataset
from tqdm import tqdm

# Import our modules
import sys
import os

utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))
models_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'models'))

if utils_path not in sys.path:
    sys.path.insert(0, utils_path)
if models_path not in sys.path:
    sys.path.insert(0, models_path)

from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from bert_training import create_training_pairs, handle_class_imbalance, add_special_tokens, tokenize_function, compute_metrics, build_gold_lookup, get_label_for_pair
from bert_extractor import preprocess_input, bert_extraction, mark_entities_full_text
from bert_model import BertRC
from relative_date_extractor import add_relative_dates

In [2]:
# Set seed for reproducibility
set_seed(42)

Data Loading

In [ ]:
# Load data
df = load_data("../data/medcat_trainer_dataset.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 5 records


In [4]:
#Add relative dates if not already added via MedCAT trainer 
if 'relative_dates_json' not in df.columns:
    df = add_relative_dates(df)
    print("Added relative dates")
else:
    print("Relative dates already present, skipping extraction")

Added relative dates


In [5]:
#Inspect df to check that relative dates have been added
#df

In [6]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 5 samples


Model & Data Preparation for Finetuning

In [13]:
# Load base model and tokenizer
model_name = "google/bert_uncased_L-2_H-128_A-2"
#model_name = "emilyalsentzer/Bio_ClinicalBERT"
base_tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Create training pairs using best approach
processed_df = create_training_pairs(samples)
print(f"\nCreated {len(processed_df)} training pairs")


Created 756 training pairs


In [19]:
# Handle class imbalance
balanced_df, class_weights = handle_class_imbalance(processed_df, method='weighted')
print(f"Class weights: {class_weights}")

Class weights: tensor([0.1164, 1.8836])


In [20]:
# Train-test split
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42, stratify=balanced_df['label'])
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

Train: 604, Test: 152


In [21]:
# Setup tokenizer with special tokens
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = add_special_tokens(tokenizer)

In [22]:
# Resize model embeddings to match new tokenizer size
base_model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30526, 128, padding_idx=0)

In [23]:
# Prepare PyTorch datasets
train_dataset = Dataset.from_pandas(train_df[['marked_text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['marked_text', 'label']])

In [24]:
# Tokenize
train_tokenized = train_dataset.map(lambda x: tokenize_function(x, tokenizer, max_length=256), batched=True)
test_tokenized = test_dataset.map(lambda x: tokenize_function(x, tokenizer, max_length=256), batched=True)

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

Map:   0%|          | 0/152 [00:00<?, ? examples/s]

In [25]:
# Set format for PyTorch
train_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

BERT Finetuning

In [26]:
# Create custom model with span pooling
model = BertRC(
    model_name=model_name,
    tokenizer=tokenizer,
    num_labels=2,
    class_weights=class_weights
)

In [27]:
# Resize model embeddings to match new tokenizer size
model.backbone.resize_token_embeddings(len(tokenizer))

Embedding(30526, 128, padding_idx=0)

In [28]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./bert_rc_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to=[],
    seed=42,
)

In [29]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

C:\Users\Satyam\AppData\Local\Temp\ipykernel_22296\2186443392.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
#Train
trainer.train()

c:\Users\Satyam\Downloads\pituitary_adenoma\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted
1,0.599000,0.985021,0.940789,0.484746,0.940789,0.912087
2,0.723700,1.422796,0.940789,0.484746,0.940789,0.912087
3,0.940500,1.560878,0.940789,0.484746,0.940789,0.912087


c:\Users\Satyam\Downloads\pituitary_adenoma\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Satyam\Downloads\pituitary_adenoma\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=228, training_loss=0.740918937482332, metrics={'train_runtime': 35.7328, 'train_samples_per_second': 50.71, 'train_steps_per_second': 6.381, 'total_flos': 0.0, 'train_loss': 0.740918937482332, 'epoch': 3.0})

In [31]:
# Evaluate on test set
eval_results = trainer.evaluate(test_tokenized)
print("\nTest Results:")
for metric, value in eval_results.items():
    if not metric.startswith('eval_'):
        continue
    clean_metric = metric.replace('eval_', '')
    print(f"{clean_metric}: {value:.4f}")

c:\Users\Satyam\Downloads\pituitary_adenoma\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Test Results:
loss: 0.9850
accuracy: 0.9408
f1_macro: 0.4847
f1_micro: 0.9408
f1_weighted: 0.9121
runtime: 0.4319
samples_per_second: 351.9570
steps_per_second: 23.1550


In [ ]:
# Save the final model
trainer.save_model("./bert_rc_final_model")
tokenizer.save_pretrained("./bert_rc_final_model")
print("\nModel saved to ./bert_rc_final_model")